In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec

indexName = "hybrid-search-langchain-pinecone"
pineconeClient = Pinecone(api_key = pinecone_api_key)

if indexName not in pineconeClient.list_indexes().names():
    pineconeClient.create_index(
        name = indexName,
        dimension = 384, 
        metric = 'dotproduct',
        spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1'),
    )

In [ ]:
index = pineconeClient.Index(indexName)
index

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-l6-v2')
embeddings

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25Encoder = BM25Encoder().default()
bm25Encoder

In [ ]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited Berlin"
]

bm25Encoder.fit(sentences)

In [ ]:
bm25Encoder.dump('bm25_values.json')
bm25Encoder = BM25Encoder().load('bm25_values.json')

In [ ]:
retriever = PineconeHybridSearchRetriever(
    embeddings = embeddings,
    sparse_encoder = bm25Encoder,
    index = index
)

retriever

In [ ]:
retriever.add_texts(
    [
        "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited Berlin"
    ]
)

In [ ]:
retriever.invoke('Which city did I visit first?')